In [2]:
import numpy as np
import pandas as pd
import xlsxwriter

In [16]:
su = pd.read_excel('../DataFiles/surveyUnits.xlsx')
cd = pd.read_excel('../DataFiles/contractsIDs.xlsx')
ch = pd.read_excel('../DataFiles/contractsHistoryIDs.xlsx')
SU = pd.read_excel("../DataFiles/Survey Units.xlsx")
CD= pd.read_excel("../DataFiles/Contracts.xlsx")
CH= pd.read_excel("../DataFiles/Contract History.xlsx")

In [17]:
su.loc[0, 'code2'] = 0
cd.loc[0, 'code2'] = 0
ch.loc[0, 'code2'] = 0

In [23]:
sus = su.sort(columns=['SR', 'bs'])
cds = cd.sort(columns=['SR', 'bs'])
chs = ch.sort(columns=['SR', 'bs'])

In [24]:
sus.index = range(len(sus))
cds.index = range(len(cds))
chs.index = range(len(chs))

In [52]:
cds.head()

,SR,BA,EA,bs,es,UID,uidp,uidt,code,bsi,uid,code2
0,2,0,0.13,0,1,1,1,1,1,1,2,0
1,2,0,0.57,0,5,1,1,1,1,3,4,1
2,2,0,0.57,0,5,1,1,1,1,3,4,0
3,2,0,0.57,0,5,1,1,1,1,3,4,0
4,2,0,0.63,0,6,1,1,1,1,3,4,1


In [53]:
chs[chs.code ==1].head()

,SR,BA,EA,bs,es,UID,uidp,uidt,code,bsi,uid,code2
0,2,0.00,0.10,0,1,1,1,1,1,1,2,0
1,2,0.00,0.12,0,1,1,1,1,1,1,2,0
2,2,0.00,0.13,0,1,1,1,1,1,1,2,0
3,2,0.00,0.07,0,0,1,1,1,1,0,1,0
4,2,0.07,0.15,0,1,1,1,1,1,1,2,1


In [56]:
sus[sus.uid == 2945].head()

,SR,BA,EA,bs,es,UID,uidp,uidt,code,bsi,uid,code2
3639,2,294.31,294.41,2943,2944,2944,2943,2944,1,2944,2945,0
3640,2,294.31,294.41,2943,2944,2944,2944,2945,1,2944,2945,0


In [31]:
def addVars(idcode):
    chs.loc[1:, 'uidp'] = np.array(chs.loc[:, idcode])[:-1]
    cds.loc[1:, 'uidp'] = np.array(cds.loc[:, idcode])[:-1]
    sus.loc[1:, 'uidp'] = np.array(sus.loc[:, idcode])[:-1]
    chs.loc[0, 'uidp'] = chs.loc[0, idcode]
    cds.loc[0, 'uidp'] = cds.loc[0, idcode]
    sus.loc[0, 'uidp'] = sus.loc[0, idcode]
    chs.loc[:(len(ch)-2), 'uidt'] = np.array(chs.loc[:, idcode])[1:]
    cds.loc[:(len(cds)-2), 'uidt'] = np.array(cds.loc[:, idcode])[1:]
    sus.loc[:(len(su)-2), 'uidt'] = np.array(sus.loc[:, idcode])[1:]
    chs.loc[(len(ch)-1), 'uidt'] = chs.loc[(len(ch)-1), idcode]
    cds.loc[(len(cds)-1), 'uidt'] = cds.loc[(len(cds)-1), idcode]
    sus.loc[(len(su)-1), 'uidt'] = sus.loc[(len(su)-1), idcode]

In [32]:
addVars('UID')

In [57]:
def refineCode(row):
    id1 = row['UID']
    id2 = row['uidp']
    id3 = row['uidt']
    if id1 == id2:
        return 1
    elif id1 == id3:
        return 1
    return 0
def refineCode2(row):
    id1 = row['uid']
    id2 = row['uidp']
    id3 = row['uidt']
    if id1 == id2:
        return 1
    elif id1 == id3:
        return 1
    return 0

In [37]:
chs.loc[:, 'code'] = chs.apply(refineCode, axis =1)
cds.loc[:, 'code'] = cds.apply(refineCode, axis =1)
sus.loc[:, 'code'] = sus.apply(refineCode, axis =1)

In [45]:
def modifyCode(row):
    code = row['code']
    bs   = row['bs']
    es   = row['es']
    if code == 1:
        return int(round((bs+es)/2))
    else:
        return bs

In [46]:
chs.loc[:, 'bsi'] = chs.apply(modifyCode, axis =1)
cds.loc[:, 'bsi'] = cds.apply(modifyCode, axis =1)
sus.loc[:, 'bsi'] = sus.apply(modifyCode, axis =1)

In [47]:
def getUID2(row):
    rd = row['SR']
    bg = row['bsi']
    code = row['code']
    if code == 1:
        res = db[((db.SR==rd) & (db.bs == bg))].UID
        if list(res) == []:
            return None
        else:
            return list(res)[0]
    else:
        return row['UID']

In [49]:
db = pd.read_excel('../DataFiles/sid.xlsx')

In [50]:
db.head()

,SR,bs,es,UID
0,2,0,1,1
1,2,1,2,2
2,2,2,3,3
3,2,3,4,4
4,2,4,5,5


In [51]:
chs.loc[:, 'uid'] = chs.apply(getUID2, axis =1)
cds.loc[:, 'uid'] = cds.apply(getUID2, axis =1)
sus.loc[:, 'uid'] = sus.apply(getUID2, axis =1)

In [58]:
addVars('uid')

In [59]:
chs.loc[:, 'code2'] = chs.apply(refineCode2, axis =1)
cds.loc[:, 'code2'] = cds.apply(refineCode2, axis =1)
sus.loc[:, 'code2'] = sus.apply(refineCode2, axis =1)

In [60]:
print len(chs[chs.code2==1]), len(chs)
print len(cds[cds.code2==1]), len(cds)
print len(sus[sus.code2==1]), len(sus)

12270 28588
41876 54196
36449 91022


In [61]:
print len(chs[chs.code==1]), len(chs)
print len(cds[cds.code==1]), len(cds)
print len(sus[sus.code==1]), len(sus)

21040 28588
50325 54196
39909 91022


In [62]:
cch = pd.merge(cds, chs, how = 'left',left_on = ['uid'], right_on = ['uid'])

In [63]:
cch.head()

,SR_x,BA_x,EA_x,bs_x,es_x,UID_x,uidp_x,uidt_x,code_x,bsi_x,...,BA_y,EA_y,bs_y,es_y,UID_y,uidp_y,uidt_y,code_y,bsi_y,code2_y
0,2,0,0.13,0,1,1,2,4,1,1,...,0.00,0.10,0,1,1,2,2,1,1,1
1,2,0,0.13,0,1,1,2,4,1,1,...,0.00,0.12,0,1,1,2,2,1,1,1
2,2,0,0.13,0,1,1,2,4,1,1,...,0.00,0.13,0,1,1,2,1,1,1,1
3,2,0,0.13,0,1,1,2,4,1,1,...,0.07,0.15,0,1,1,1,4,1,1,0
4,2,0,0.13,0,1,1,2,4,1,1,...,0.00,0.11,0,1,1,4,2,1,1,1


In [64]:
cchs = cch.dropna(subset = ['BA_y', 'EA_y'])

In [66]:
print len(cchs), len(cds)

1516502 54196


In [67]:
cdc = cds.drop_duplicates(subset = ['uid'])
chc = chs.drop_duplicates(subset = ['uid'])
suc = sus.drop_duplicates(subset = ['uid'])

In [69]:
print len(cds), len(cdc)
print len(chs), len(chc)
print len(sus), len(suc)

54196 16448
28588 15495
91022 69385


In [70]:
m1 = pd.merge(suc, chc, how = 'left', left_on =['uid'], right_on =['uid'])

In [76]:
m2.columns

Index([   u'SR_x',    u'BA_x',    u'EA_x',    u'bs_x',    u'es_x',   u'UID_x',
        u'uidp_x',  u'uidt_x',  u'code_x',   u'bsi_x',     u'uid', u'code2_x',
          u'SR_y',    u'BA_y',    u'EA_y',    u'bs_y',    u'es_y',   u'UID_y',
        u'uidp_y',  u'uidt_y',  u'code_y',   u'bsi_y', u'code2_y',      u'SR',
            u'BA',      u'EA',      u'bs',      u'es',     u'UID',    u'uidp',
          u'uidt',    u'code',     u'bsi',   u'code2'],
      dtype='object')

In [72]:
sch = m1.dropna(subset = [ u'bs_y',    u'es_y'])

In [73]:
len(sch)

15385

In [75]:
m2 = pd.merge(sch, cdc, how = 'left', left_on =['uid'], right_on =['uid'])

In [78]:
ds = m2.dropna(subset = [ u'bs',    u'es'])

In [79]:
len(ds)

10779

In [80]:
ds.head()

,SR_x,BA_x,EA_x,bs_x,es_x,UID_x,uidp_x,uidt_x,code_x,bsi_x,...,BA,EA,bs,es,UID,uidp,uidt,code,bsi,code2
0,2,0.00,0.10,0,1,1,2,2,1,1,...,0.00,0.13,0,1,1,2,4,1,1,1
1,2,0.00,0.07,0,0,1,2,2,1,0,...,0.00,0.04,0,0,1,2,1,1,0,1
2,2,0.13,0.23,1,2,2,2,2,1,2,...,0.00,0.49,0,4,1,9,3,1,2,1
3,2,0.23,0.35,2,3,3,3,4,1,3,...,0.00,0.57,0,5,1,2,4,1,3,1
4,2,0.35,0.45,3,4,4,4,5,1,4,...,0.39,0.48,3,4,4,15,4,1,4,0


In [88]:
# sch.head()

In [89]:
sud = pd.merge(SU, suc, how = 'left', left_on =['SR', 'Begin ARM', 'End ARM'], right_on =['SR', 'BA', 'EA'])

In [94]:
sut = sud.dropna(subset = ['bs', 'es'])
surveyUnits = sut.drop_duplicates(subset = ['uid'])

In [95]:
print len(surveyUnits)
print len(surveyUnits) == len(suc)

69385
True


In [96]:
chd = pd.merge(CH, chc, how = 'left', left_on =['SR', 'Begin ARM', 'End ARM'], right_on =['SR', 'BA', 'EA'])
cht = chd.dropna(subset = ['bs', 'es'])
contractsHistory = cht.drop_duplicates(subset = ['uid'])

In [97]:
print len(contractsHistory)
print len(contractsHistory) == len(chc)

15495
True


In [98]:
cdd = pd.merge(CD, cdc, how = 'left', left_on =['SR', 'Begin ARM', 'End ARM'], right_on =['SR', 'BA', 'EA'])
cdt = cdd.dropna(subset = ['bs', 'es'])
contracts = cdt.drop_duplicates(subset = ['uid'])

In [99]:
print len(contracts)
print len(contracts) == len(cdc)

16448
True


In [100]:
contracts.to_excel('../DataFiles/contractsR.xlsx')
contractsHistory.to_excel('../DataFiles/contractsHistR.xlsx')
surveyUnits.to_excel('../DataFiles/surveyUnitsR.xlsx')